In [2]:

import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [4]:
df=pd.read_csv('all_season_details.csv')

C:\Users\govinda prasad\AppData\Local\Temp\ipykernel_12380\2416424084.py:1: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('all_season_details.csv')


In [6]:
df.drop(['comment_id','shortText','text','preText','postText'],axis=1,inplace=True)


In [8]:
df.loc[(df['match_id'].isin([1359496,1359538])) & (df['season'].isnull()), 'season'] = 2023


In [10]:
df.drop_duplicates(inplace=True)


In [12]:
new_df = df[['season', 'match_id', 'home_team', 'away_team', 'current_innings',
                      'innings_id', 'over', 'ball', 'runs', 'wicket_id', 
                      'wkt_batsman_name', 'wkt_batsman_runs']].copy()

new_df['cumulative_runs'] = new_df.groupby(['match_id', 'current_innings'])['runs'].cumsum()

new_df['run_rate'] = new_df['cumulative_runs'] / (new_df['over'] + 0.1)

new_df['wickets_lost'] = new_df.groupby(['match_id', 'current_innings'])['wkt_batsman_runs']\
                                                 .transform(lambda x: (x >= 0).cumsum())
new_df.fillna(0, inplace=True)


In [14]:
unique_home_teams = new_df['home_team'].unique().tolist()

home_teams_df = pd.DataFrame(unique_home_teams, columns=['Teams'])

print(home_teams_df)


    Teams
0      GT
1    PBKS
2     LSG
3     SRH
4     RCB
5     CSK
6      DC
7      RR
8     KKR
9      MI
10   KXIP
11    RPS
12     GL
13    PWI
14  Kochi


In [42]:
# Create a dictionary for mapping
team_mapping = {'GT': 0, 'PBKS': 1, 'LSG': 2, 'SRH': 3,'RCB': 4, 'CSK': 5, 'DC': 6, 'RR': 7,
                'KKR': 8, 'MI': 9, 'KXIP': 10, 'RPS': 11,'GL': 12, 'PWI': 13, 'KOCHI': 14}

# Apply the mapping to 'home_team' and 'away_team'
new_df['home_team_encoded'] = new_df['home_team'].map(team_mapping)
new_df['away_team_encoded'] = new_df['away_team'].map(team_mapping)

new_df['current_innings'] = new_df['current_innings'].map(team_mapping)
new_df.fillna(0, inplace=True)


In [44]:
X = new_df[['home_team_encoded','away_team_encoded','current_innings','over', 'ball', 'run_rate','wickets_lost']]
y = new_df['cumulative_runs']

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [48]:
X_train

,home_team_encoded,away_team_encoded,current_innings,over,ball,run_rate,wickets_lost
199033,13.0,8.0,0.0,12,4,5.537190,3
237647,6.0,3.0,0.0,17,1,8.596491,2
10462,0.0,6.0,0.0,1,5,4.545455,1
39476,7.0,4.0,0.0,8,4,8.518519,0
137929,3.0,5.0,0.0,9,2,6.153846,2
...,...,...,...,...,...,...,...
119880,8.0,3.0,0.0,10,6,7.029703,3
103695,3.0,9.0,0.0,12,4,8.016529,1
131933,10.0,4.0,0.0,6,1,8.196721,2
146868,8.0,3.0,0.0,18,6,7.182320,5


In [84]:
rf_model=RandomForestRegressor()
rf_model.fit(X_train,y_train)

y_rf_simple_pred=rf_model.predict(X_test)
rf_mse=mean_squared_error(y_test,y_rf_simple_pred)
rf_r2=r2_score(y_test,y_rf_simple_pred)

print('MSE score: ',rf_mse)
print('R2 score: ',rf_r2)

MSE score:  0.0679065367965368
R2 score:  0.999972790846467


In [85]:
new_data = pd.DataFrame({
    'home_team_encoded': [1],
    'away_team_encoded': [5],
    'current_innings':[1],
    'over': [12],  
    'ball': [2],      
    'run_rate': [10],        
    'wickets_lost': [2]          
})

In [86]:
predict_runs=rf_model.predict(new_data)
print(f"Predicted Cumulative Runs: {predict_runs[0]}")

Predicted Cumulative Runs: 121.0


In [87]:
print('Linear  regression')

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures


poly_model = LinearRegression()
poly_model.fit(X_train, y_train)

pred = poly_model.predict(X_test)

lr_mse=mean_squared_error(y_test,pred)
print('MSE score: ',lr_mse)
lr_r2=r2_score(y_test,pred)
print('R2 score: ',lr_r2)

Linear  regression
MSE score:  266.80583330355216
R2 score:  0.8930948149570086


In [88]:
print('Polynomial  regression')

poly = PolynomialFeatures(degree=2) 
X_polya = poly.fit_transform(X_train)
X_polyb=poly.fit_transform(X_test)


poly_model = LinearRegression()
poly_model.fit(X_polya, y_train)


pred = poly_model.predict(X_polyb)

pr_mse=mean_squared_error(y_test,pred)
print('MSE score: ',pr_mse)

pr_r2=r2_score(y_test,pred)
print('R2 score: ',pr_r2)


Polynomial  regression
MSE score:  4.181228379056165e-24
R2 score:  1.0


In [89]:
print('Decision Tree  regression')

from sklearn.tree import DecisionTreeRegressor

tree_model = DecisionTreeRegressor(max_depth=3)
tree_model.fit(X_train, y_train)

pred = tree_model.predict(X_test)

tree_mse=mean_squared_error(y_test,pred)
print('MSE score: ',tree_mse)

tree_r2=r2_score(y_test,pred)
print('R2 score: ',tree_r2)

Decision Tree  regression
MSE score:  327.7749015281932
R2 score:  0.8686654033517592


In [100]:
from sklearn.ensemble import GradientBoostingRegressor

gb_model = GradientBoostingRegressor(random_state=42)
gb_model.fit(X_train, y_train)

y_pred_gb = gb_model.predict(X_test)

mse_gb = mean_squared_error(y_test, y_pred_gb)
r2_gb = r2_score(y_test, y_pred_gb)

print("Gradient Boosting Regressor:")
print("MSE score: ",mse_gb)
print("R2 score: ",r2_gb)

Gradient Boosting Regressor:
MSE score:  3.469246076764565
R2 score:  0.9986099239690377


In [102]:
from xgboost import XGBRegressor


xgb_model = XGBRegressor(random_state=42)
xgb_model.fit(X_train, y_train)

y_pred_xgb = xgb_model.predict(X_test)

mse_xgb = mean_squared_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print("XGBoost Regressor:")
print("MSE score: ",mse_xgb)
print("R2 score: ",r2_xgb)

XGBoost Regressor:
MSE score:  1.365125512728276
R2 score:  0.9994530081748962
